## AWS Database Review - PROD Accounts

In [3]:
Import-Module AWS.Tools.Common
Import-Module AWS.Tools.DynamoDBv2
Import-Module AWS.Tools.RDS


$Profiles   = Get-AWSCredentials -ListProfileDetail | Where-Object { $_.ProfileName -like 'DevOps*' }
$Profiles

function Get-AWSDBInformation {
    [CmdletBinding()]
    param (

        $Profiles,
        $Regions,
        $RegexGroup             = 'AccountNumber',
        $RegexGroupAccountName  = 'AccountName',
        $RegexAccountNumber     = "(?<$RegexGroup>\d+):db.+$",
        $RegexAccountNumberDDb  = "(?<$RegexGroup>\d+):table.+$" ,
        $RegexAccountName       = "(?<PreFix>DevOps-)(?<$RegexGroupAccountName>.+)"
    )
    
    begin {

        $Summary    = @()
        
    }
    
    process {

        $Summary    = @()
        foreach ($Profile in $Profiles) {

            foreach ($Region in $Regions) {
                
                $RDSInstances   = Get-RDSDBInstance -ProfileName $Profile.ProfileName -Region $Region
                $ddbTables      = Get-DDBTableList -ProfileName $Profile.ProfileName -Region $Region

                switch ([string]::IsNullOrEmpty($RDSInstances)) {
                    $true { 
                        if ($ddbTables) {
                            
                            $Table = $ddbTables | Select-Object -First 1
                            $ddbTable = Get-DDBTable -TableName $Table -ProfileName $Profile.ProfileName -Region $Region
                            $AccountID  = (($ddbTable.TableArn | Select-String -Pattern $RegexAccountNumberDDb).Matches.Groups | Where-Object { $_.Name -eq $RegexGroup }).Value
                        }else {

                            $AccountID = 0
                        }
                    }
                    $false {

                        $RDSInstance   = $RDSInstances | Select-Object -First 1
                        $AccountID  = (($RDSInstance.DBInstanceArn | Select-String -Pattern $RegexAccountNumber).Matches.Groups | Where-Object { $_.Name -eq $RegexGroup }).Value
                    }
                    default {

                        $AccountID = 0
                    }
                }

                $Sum = [PSCustomObject]@{

                    Account             = (($Profile.ProfileName | select-string -Pattern $RegexAccountName).Matches.Groups | Where-Object { $_.Name -eq $RegexGroupAccountName }).Value
                    AccountID           = $AccountID
                    Region              = $Region
                    RDSInstances        = ($RDSInstances.DBInstanceIdentifier | Measure-Object).Count
                    DynamoDBTables      = $ddbTables.Count
                    
                }

                $condition = $Sum.RDSInstances + $Sum.DynamoDBTables
                if ($condition -gt 0) {
                    $Summary += $Sum
                }
            }

        }
        
    }
    
    end {

        $Summary
        
    }
}


ProfileName                      StoreTypeName         ProfileLocation
-----------                      -------------         ---------------
DevOps-Apex-PROD                 SharedCredentialsFile /Users/conrad.gauntlett/.aws/credentials
DevOps-Owner-DK-Prod             SharedCredentialsFile /Users/conrad.gauntlett/.aws/credentials
DevOps-Owner-Services-PROD       SharedCredentialsFile /Users/conrad.gauntlett/.aws/credentials
DevOps-AVR-Guest-Experience-PROD SharedCredentialsFile /Users/conrad.gauntlett/.aws/credentials
DevOps-AVR-Guest-Payments-PROD   SharedCredentialsFile /Users/conrad.gauntlett/.aws/credentials
DevOps-AGE-Checkout-PROD         SharedCredentialsFile /Users/conrad.gauntlett/.aws/credentials
DevOps-AVR-PostBook-PROD         SharedCredentialsFile /Users/conrad.gauntlett/.aws/credentials
DevOps-AVR-Pricing-PROD          SharedCredentialsFile /Users/conrad.gauntlett/.aws/credentials
DevOps-Novosol-Awaze-PROD        SharedCredentialsFile /Users/conrad.gauntlett/.aws/crede

In [ ]:
$Regions    = "eu-central-1" , "eu-west-2" , "eu-west-1", "us-east-1"
$Summary    = Get-AWSDBInformation $Profiles $Regions
$Summary    | Format-Table -AutoSize
# $Summary    | export-csv -Path '/Users/conrad.gauntlett/WorkArea/Temp/AWSDBInformation.csv' -NoTypeInformation -Force